# Extraccion, transformacion y carga de datos

### Importamos librerias

In [1]:
import requests
from bs4 import BeautifulSoup
from modulos.data.webdriver import iniciar_chrome
from modulos.utils.wait import *
import pandas as pd

In [2]:
# Para recargar las librerias locales
%load_ext autoreload
%autoreload 2

## Verifiquemos el sitio web


Definimos url de la pagina a recabar informacion, y el user agent (requisito)

In [3]:
url = "https://www.supercasas.com/buscar/?do=1&ObjectType=123&PriceType=401&Locations=10005&PriceFrom=0.00&PriceTo=200000.00&SizeLotFrom=0&SizeLotTo=25000&OrderDirection=DESC&OrderColumn=Price&PagingPageSkip=0"
h = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"}

Verificamos el funcionamiento


In [4]:
try:
    res = requests.get(url=url,headers=h, timeout= 10)
except TimeoutError:
    print("La pagina no arroja resultados")

In [5]:

if res.status_code <400 and res.ok == True:
    print("La solicitud ha sido aprovada con exito")
else:
    print(f"Ha fallado\n codigo:{res.status_code}")

La solicitud ha sido aprovada con exito


Abrimos el codigo html para comprobar si estamos ubicados donde deseamos

In [6]:
with open("../references/codigo_200.html","w",encoding="utf-8") as f:
    f.write(res.text)

Al hacer Preview al archivo html podemos ver que estamos bien ubicados
![image.png](img/PreviewHTML.png)

## Extraccion

Iniciamos el bucle "for" para recolectar todas las paginas de oferta del portal, esto podemos hacerlo creando una lista con las url de las 41 paginas de ofertas 



In [26]:
# URL base con el marcador {page} donde se insertará el número de página
url_base = "https://www.supercasas.com/buscar/?do=1&ObjectType=123&PriceType=401&Locations=10005&PriceFrom=0.00&PriceTo=200000.00&SizeLotFrom=0&SizeLotTo=25000&OrderColumn=Price&PagingPageSkip={page}"

# Lista para almacenar las URLs
ofertas = []

# Bucle para crear las URLs cambiando el valor de 'PagingPageSkip' de 0 a 41
for i in range(42):
    # Reemplaza {page} con el valor de i
    url = url_base.format(page=i)
    # Agrega la URL generada a la lista
    ofertas.append(url)

# Imprime la lista de URLs
print(ofertas)

['https://www.supercasas.com/buscar/?do=1&ObjectType=123&PriceType=401&Locations=10005&PriceFrom=0.00&PriceTo=200000.00&SizeLotFrom=0&SizeLotTo=25000&OrderColumn=Price&PagingPageSkip=0', 'https://www.supercasas.com/buscar/?do=1&ObjectType=123&PriceType=401&Locations=10005&PriceFrom=0.00&PriceTo=200000.00&SizeLotFrom=0&SizeLotTo=25000&OrderColumn=Price&PagingPageSkip=1', 'https://www.supercasas.com/buscar/?do=1&ObjectType=123&PriceType=401&Locations=10005&PriceFrom=0.00&PriceTo=200000.00&SizeLotFrom=0&SizeLotTo=25000&OrderColumn=Price&PagingPageSkip=2', 'https://www.supercasas.com/buscar/?do=1&ObjectType=123&PriceType=401&Locations=10005&PriceFrom=0.00&PriceTo=200000.00&SizeLotFrom=0&SizeLotTo=25000&OrderColumn=Price&PagingPageSkip=3', 'https://www.supercasas.com/buscar/?do=1&ObjectType=123&PriceType=401&Locations=10005&PriceFrom=0.00&PriceTo=200000.00&SizeLotFrom=0&SizeLotTo=25000&OrderColumn=Price&PagingPageSkip=4', 'https://www.supercasas.com/buscar/?do=1&ObjectType=123&PriceType=401

In [27]:
# Lista para guardar los enlaces de las ofertas
enlaces_ofertas = []

# Bucle para recorrer cada URL
for url in ofertas:
    # Realiza una solicitud GET a la URL
    response = requests.get(url, headers=h)
    pausa()
    # Verifica si la solicitud fue exitosa
    if response.status_code < 300:
        # Analiza el contenido HTML de la página
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Encuentra el contenedor de resultados
        contenedor_resultados = soup.find('div', id='bigsearch-results-inner-results')
        
        if contenedor_resultados:
            # Encuentra todas las etiquetas <li> dentro del contenedor
            ofertas = contenedor_resultados.find_all('li')
            # Recorre cada oferta y extrae el enlace del bloque <a>
            for oferta in ofertas:
                enlace = oferta.find('a', href=True)
                if enlace:
                    # Obtiene el atributo href y lo agrega a la lista
                    url_completo = "https://supercasas.com" + enlace['href']
                    enlaces_ofertas.append(url_completo)
                    



In [30]:
# Imprime la lista de enlaces de las ofertas
print(enlaces_ofertas[:3]) # observamos que tenemos los enlaces funcionales, primeras 3



['https://supercasas.com/apartamentos-alquiler-el-millon/1367994/', 'https://supercasas.com/apartamentos-alquiler-zona-universitaria/1365908/', 'https://supercasas.com/apartamentos-venta-y-alquiler-av--anacaona/1358370/']


In [31]:
print(enlaces_ofertas[-3:]) #ultimas 3

['https://supercasas.com/apartamentos-alquiler-bella-vista/1365839/', 'https://supercasas.com/apartamentos-alquiler-los-cacicazgos/1365838/', 'https://supercasas.com/apartamentos-alquiler-gazcue/1365830/']


Comprobemos el tamaño que deberia ser la lista para verificar si no hay inciertos

Segun calculos manuales si cada pagina contiene 24 ofertas y existen 42 paginas, tenemos que es 24 (ofertas) * 41 (paginas) + 16 que contiene la ultima pagina = 1000 ofertas

In [32]:
#Verifiquemos si enlaces_ofertas contiene 1000 elementos

if len(enlaces_ofertas) == 1000:
    print("Si tiene 1000 ofertas")
else:
    print(f"No tiene 1000 ofertas, tiene {len(enlaces_ofertas)} ofertas")

Si tiene 1000 ofertas


Ya tenemos extraido cada oferta de la pagina, creemos un dataset con la informacion especifica de cada oferta:
metros cuadrados, sector, cantidad de baños, habitacion y parqueos, 

In [ ]:
# Lista para almacenar los datos de cada oferta
data = []

# Bucle para recorrer cada enlace de la oferta
for enlace in enlaces_ofertas:
    # Realiza una solicitud GET al enlace de la oferta
    response = requests.get(enlace,headers=h)
    
    # Verifica si la solicitud fue exitosa
    if response.status_code < 300:
        # Analiza el contenido HTML de la página
        soup = BeautifulSoup(response.text, 'html.parser')
        
        
        
        # Añade la información extraída como un registro en el dataset
        """data.append({
            'Sector': sector,
            'Precio': precio,
            'Habitaciones': habitaciones,
            'Baños': baños,
            'Parqueos': parqueos,
            'Comodidades': comodidades,
            'Observaciones': observaciones
        }) """